In [9]:
!pipx install segmentation_models_pytorch

'segmentation-models-pytorch' already seems to be installed. Not modifying
existing installation in
'/home/hardik/.local/share/pipx/venvs/segmentation-models-pytorch'. Pass
'--force' to force installation.


In [10]:
import os
import numpy as np
import joblib as jlb
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import segmentation_models_pytorch as smp
from sklearn.model_selection import train_test_split
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm

# ==================== Configuration ====================
CONFIG = {
    'ims_path': r'/home/hardik/Desktop/python_intern/archive (1)/ims.np',
    'mas_path': r'/home/hardik/Desktop/python_intern/archive (1)/mas.np',
    'model_save_path': r'/home/hardik/Desktop/python_intern/unet_segmentation_model.pth',
    'image_size': (256, 256),
    'batch_size': 8,
    'num_epochs': 50,
    'learning_rate': 1e-4,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'num_workers': 4, # Adjust based on your system
    'encoder': 'resnet34',
    'encoder_weights': 'imagenet',
}

print(f"Using device: {CONFIG['device']}")

ModuleNotFoundError: No module named 'segmentation_models_pytorch'

## 1. Data Preprocessing and Custom Dataset 💾
Defines the function to convert binary masks into a 3-class segmentation problem (Background: 0, Residential: 1, Commercial: 2) and the custom PyTorch `Dataset`.

In [ ]:
def create_multiclass_masks(masks, building_counts):
    """
    Convert binary masks to 3-class segmentation:
    0 = background, 1 = residential, 2 = commercial (high density)
    """
    # Determine the threshold for 'Commercial' density (e.g., top 10% density)
    if len(building_counts) == 0: return np.zeros_like(masks, dtype=np.int64)
    threshold = np.percentile(building_counts, 90)
    multiclass_masks = np.zeros_like(masks, dtype=np.int64)
    
    for i in range(len(masks)):
        mask = masks[i]
        if building_counts[i] > threshold:
            # Commercial (Class 2)
            multiclass_masks[i] = np.where(mask > 0, 2, 0)
        else:
            # Residential (Class 1)
            multiclass_masks[i] = np.where(mask > 0, 1, 0)
    
    return multiclass_masks


# ==================== Custom Dataset ====================
class SatelliteDataset(Dataset):
    def __init__(self, images, masks, transform=None):
        self.images = images
        self.masks = masks
        self.transform = transform
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image = self.images[idx].astype(np.float32)
        mask = self.masks[idx].astype(np.int64)
        
        # Normalize image to [0, 1]
        if image.max() > 1.0:
            image = image / 255.0
        
        if self.transform:
            # Albumentations expects HWC format, float32 image
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image'] # Already normalized and to Tensor (CHW)
            mask = augmented['mask'] # Tensor (HW)
        
        return image, mask

## 2. Augmentation Pipelines 🤸
Defines the transformation pipelines for training (with augmentation) and validation (without augmentation).

In [ ]:
def get_training_augmentation(image_size):
    # Standard image normalization values for ImageNet pre-trained models
    mean=(0.485, 0.456, 0.406)
    std=(0.229, 0.224, 0.225)
    
    return A.Compose([
        A.Resize(height=image_size[0], width=image_size[1]),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5, border_mode=0),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
        A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
        A.Normalize(mean=mean, std=std), # Applies Z-score normalization
        ToTensorV2(), # Converts to PyTorch Tensor and changes layout to CHW
    ])

def get_validation_augmentation(image_size):
    mean=(0.485, 0.456, 0.406)
    std=(0.229, 0.224, 0.225)
    
    return A.Compose([
        A.Resize(height=image_size[0], width=image_size[1]),
        A.Normalize(mean=mean, std=std),
        ToTensorV2(),
    ])

## 3. Loss Function and Metrics 📊
Implements a custom multiclass **Dice Loss** and combines it with **Cross-Entropy** for the final objective function. Also defines the **IoU** metric.

In [ ]:
class DiceLoss(nn.Module):
    def __init__(self, smooth=1.0):
        super(DiceLoss, self).__init__()
        self.smooth = smooth
    
    def forward(self, pred, target):
        # pred is logits (B, C, H, W)
        # target is class indices (B, H, W)
        pred = torch.softmax(pred, dim=1)
        
        # Convert target to one-hot encoding (B, H, W, C) -> (B, C, H, W)
        target_one_hot = torch.nn.functional.one_hot(target, num_classes=pred.shape[1])
        target_one_hot = target_one_hot.permute(0, 3, 1, 2).float()
        
        # Compute Dice for each class (Intersection and Union summed over H and W)
        intersection = (pred * target_one_hot).sum(dim=(2, 3))
        union = pred.sum(dim=(2, 3)) + target_one_hot.sum(dim=(2, 3))
        
        # Dice formula: 2*I / (U + I)
        dice = (2. * intersection + self.smooth) / (union + self.smooth)
        
        # Return 1 - mean Dice score across all classes in the batch
        return 1 - dice.mean()

class CombinedLoss(nn.Module):
    def __init__(self, alpha=0.5):
        super(CombinedLoss, self).__init__()
        self.alpha = alpha
        # CrossEntropyLoss expects logits (pred) and class indices (target)
        self.ce_loss = nn.CrossEntropyLoss()
        self.dice_loss = DiceLoss()
    
    def forward(self, pred, target):
        ce = self.ce_loss(pred, target)
        dice = self.dice_loss(pred, target)
        # Weighted sum of losses
        return self.alpha * ce + (1 - self.alpha) * dice


# ==================== Metrics ====================
def calculate_iou(pred, target, num_classes=3):
    """Calculate Intersection over Union for each class"""
    # Convert logits to class predictions
    pred = torch.argmax(pred, dim=1)
    ious = []
    
    for cls in range(num_classes):
        pred_cls = (pred == cls)
        target_cls = (target == cls)
        
        # Intersection and Union
        intersection = (pred_cls & target_cls).sum().float()
        union = (pred_cls | target_cls).sum().float()
        
        # IoU formula: I / U
        iou = (intersection + 1e-6) / (union + 1e-6) # Add epsilon for stability
        ious.append(iou.item())
    
    return ious

## 4. Training and Validation Loops ⚙️
Defines the functions to execute one epoch of training and validation.

In [ ]:
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    total_iou = [0, 0, 0]
    
    pbar = tqdm(dataloader, desc='Training')
    for images, masks in pbar:
        images = images.to(device)
        masks = masks.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        ious = calculate_iou(outputs, masks)
        total_iou = [total_iou[i] + ious[i] for i in range(3)]
        
        pbar.set_postfix({'loss': f'{loss.item():.4f}', 'mean_iou': f'{np.mean(ious):.4f}'})
    
    avg_loss = total_loss / len(dataloader)
    avg_iou = [iou / len(dataloader) for iou in total_iou]
    
    return avg_loss, avg_iou

def validate_epoch(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    total_iou = [0, 0, 0]
    
    with torch.no_grad():
        pbar = tqdm(dataloader, desc='Validation')
        for images, masks in pbar:
            images = images.to(device)
            masks = masks.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, masks)
            
            total_loss += loss.item()
            ious = calculate_iou(outputs, masks)
            total_iou = [total_iou[i] + ious[i] for i in range(3)]
            
            pbar.set_postfix({'loss': f'{loss.item():.4f}', 'mean_iou': f'{np.mean(ious):.4f}'})
    
    avg_loss = total_loss / len(dataloader)
    avg_iou = [iou / len(dataloader) for iou in total_iou]
    
    return avg_loss, avg_iou

## 5. Main Training Pipeline Execution 🚀
Loads data, splits into train/val sets, initializes the model, runs the training loop, and visualizes results.

In [ ]:
def main():
    # Load data
    print("Loading data...")
    try:
        ims = jlb.load(CONFIG['ims_path'])
        mas = jlb.load(CONFIG['mas_path'])
    except FileNotFoundError:
        print("Error: Data files not found. Check 'ims_path' and 'mas_path' in CONFIG.")
        return
    
    ims = np.array(ims)
    mas = np.array(mas)
    
    # Normalize masks shape (remove last channel if present)
    if mas.ndim == 4 and mas.shape[-1] == 1:
        mas = mas[..., 0]
    
    print(f"Images shape: {ims.shape}")
    print(f"Masks shape: {mas.shape}")
    
    # Create multiclass masks
    print("Creating multiclass masks...")
    # Sum of non-zero pixels (buildings) for each image
    building_counts = np.sum(mas > 0, axis=(1, 2))
    multiclass_masks = create_multiclass_masks(mas, building_counts)
    
    class_counts = [np.sum(multiclass_masks == i) for i in range(3)]
    print(f"Class distribution (Pixel Count) - Background: {class_counts[0]}, Residential (1): {class_counts[1]}, Commercial (2): {class_counts[2]}")
    
    # Split data
    X_train, X_val, y_train, y_val = train_test_split(
        ims, multiclass_masks, test_size=0.2, random_state=42
    )
    
    print(f"Train samples: {len(X_train)}, Val samples: {len(X_val)}")
    
    # Create datasets and dataloaders
    train_dataset = SatelliteDataset(
        X_train, y_train, 
        transform=get_training_augmentation(CONFIG['image_size'])
    )
    val_dataset = SatelliteDataset(
        X_val, y_val, 
        transform=get_validation_augmentation(CONFIG['image_size'])
    )
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=CONFIG['batch_size'], 
        shuffle=True, 
        num_workers=CONFIG['num_workers'],
        pin_memory=True
    )
    val_loader = DataLoader(
        val_dataset, 
        batch_size=CONFIG['batch_size'], 
        shuffle=False, 
        num_workers=CONFIG['num_workers'],
        pin_memory=True
    )
    
    # Create model
    print("Creating model...")
    model = smp.Unet(
        encoder_name=CONFIG['encoder'],
        encoder_weights=CONFIG['encoder_weights'],
        in_channels=3,
        classes=3  # 0: background, 1: residential, 2: commercial
    )
    model = model.to(CONFIG['device'])
    
    # Loss and optimizer
    criterion = CombinedLoss(alpha=0.5)
    optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG['learning_rate'])
    # Scheduler to dynamically adjust learning rate
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', patience=5, factor=0.5, verbose=True
    )
    
    # Training loop
    print("Starting training...")
    best_val_loss = float('inf')
    history = {'train_loss': [], 'val_loss': [], 'train_iou': [], 'val_iou': []}
    
    for epoch in range(CONFIG['num_epochs']):
        print(f"\nEpoch {epoch+1}/{CONFIG['num_epochs']}")
        
        train_loss, train_iou = train_epoch(model, train_loader, criterion, optimizer, CONFIG['device'])
        val_loss, val_iou = validate_epoch(model, val_loader, criterion, CONFIG['device'])
        
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_iou'].append(train_iou)
        history['val_iou'].append(val_iou)
        
        print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
        print(f"Train IoU - BG: {train_iou[0]:.4f}, Res: {train_iou[1]:.4f}, Com: {train_iou[2]:.4f}")
        print(f"Val IoU - BG: {val_iou[0]:.4f}, Res: {val_iou[1]:.4f}, Com: {val_iou[2]:.4f}")
        
        scheduler.step(val_loss)
        
        # Save best model based on validation loss
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': val_loss,
                'config': CONFIG
            }, CONFIG['model_save_path'])
            print(f"✅ Model saved with best val_loss: {val_loss:.4f}")
    
    print("\n\n--- Training History Plots ---")
    
    # Plot training history
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')
    
    plt.subplot(1, 3, 2)
    train_iou_bg = [iou[0] for iou in history['train_iou']]
    val_iou_bg = [iou[0] for iou in history['val_iou']]
    plt.plot(train_iou_bg, label='Train IoU BG')
    plt.plot(val_iou_bg, label='Val IoU BG')
    plt.xlabel('Epoch')
    plt.ylabel('IoU')
    plt.legend()
    plt.title('Background IoU')
    
    plt.subplot(1, 3, 3)
    train_iou_res = [iou[1] for iou in history['train_iou']]
    train_iou_com = [iou[2] for iou in history['train_iou']]
    val_iou_res = [iou[1] for iou in history['val_iou']]
    val_iou_com = [iou[2] for iou in history['val_iou']]
    plt.plot(train_iou_res, label='Train Residential')
    plt.plot(train_iou_com, label='Train Commercial')
    plt.plot(val_iou_res, label='Val Residential')
    plt.plot(val_iou_com, label='Val Commercial')
    plt.xlabel('Epoch')
    plt.ylabel('IoU')
    plt.legend()
    plt.title('Class-wise IoU')
    
    plt.tight_layout()
    plt.savefig('/home/hardik/Desktop/python_intern/training_history.png')
    plt.show()
    
    print("\n--- Prediction Visualization ---")
    
    # Visualize predictions
    model.eval()
    with torch.no_grad():
        # Get one batch from the validation loader
        sample_images, sample_masks = next(iter(val_loader))
        sample_images = sample_images.to(CONFIG['device'])
        predictions = model(sample_images)
        predictions = torch.argmax(predictions, dim=1).cpu().numpy()
    
    # Denormalize images for visualization (reverse ImageNet normalization)
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    
    n_samples = min(4, len(sample_images))
    fig, axes = plt.subplots(n_samples, 3, figsize=(12, 4*n_samples))
    # Ensure axes is a 2D array even for n_samples=1
    if n_samples == 1: axes = np.expand_dims(axes, 0)
    
    for i in range(n_samples):
        img = sample_images[i].cpu().numpy().transpose(1, 2, 0) # CHW -> HWC
        # Denormalization
        img = std * img + mean
        img = np.clip(img, 0, 1) # Clip to valid image range
        
        # Use a colormap suitable for discrete classes (0, 1, 2)
        cmap = plt.cm.get_cmap('tab10', 3)
        
        axes[i, 0].imshow(img)
        axes[i, 0].set_title('Input Image')
        axes[i, 0].axis('off')
        
        axes[i, 1].imshow(sample_masks[i].cpu().numpy(), cmap=cmap, vmin=0, vmax=2)
        axes[i, 1].set_title('Ground Truth')
        axes[i, 1].axis('off')
        
        axes[i, 2].imshow(predictions[i], cmap=cmap, vmin=0, vmax=2)
        axes[i, 2].set_title('Prediction')
        axes[i, 2].axis('off')
        
    # Add a custom legend for classes
    handles = [plt.Line2D([0], [0], marker='o', color='w', label=label, 
                          markerfacecolor=cmap(i), markersize=10) for i, label in enumerate(['Background', 'Residential', 'Commercial'])]
    fig.legend(handles=handles, loc='upper right', bbox_to_anchor=(1.15, 0.9))
    
    plt.tight_layout(rect=[0, 0, 1.05, 1])
    plt.savefig('/home/hardik/Desktop/python_intern/predictions.png')
    plt.show()
    
    print("\n✅ Training complete!")

if __name__ == '__main__':
    main()